<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/Json_import_location_split_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
#  this code is perfect, do no change
#  it loads the json, adds jon numbers
#  saves back down for later use
#  !!!!!DO NOT CHANGE!!!!!

import json
import pandas as pd
import re

# Assuming 'resume.json' is in a directory called 'resumes' in the same parent directory as your Python script
json_file_path1 = "/content/CV_parsed_by_CV_Parser_Premium-23.json"  # Relative path

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path1, 'r') as file:
    resume_data = json.load(file)

# Labeling the job entries
job_count = 1
for experience in resume_data['profile']['professional_experiences']:
    experience['job_label'] = f'Job_{job_count}'
    job_count += 1

# Save the modified data back to the JSON file
output_json_path = "/content/jobslabeled.json"
with open(output_json_path, 'w') as output_file:
    json.dump(resume_data, output_file, indent=2)

print(f"Job labeling completed. Labeled data saved to {output_json_path}")

# Now 'resume_data' contains the contents of the JSON file
# print(json.dumps(resume_data, indent=2))

# Load df1 from the Excel file
df1_path = "/content/world_cities_countries.csv"
df1 = pd.read_csv(df1_path)


def split_and_create_columns(location):
    if pd.notna(location):
        substrings = re.split(r'[^\w]+', location)
        substrings = [entry.strip() for entry in substrings]
        substrings = [remove_special_terms(entry) for entry in substrings]

        if any("justice" in entry.lower() or "precinct" in entry.lower() for entry in substrings):
            substrings.append("new zealand")

        if any("christchurch" in entry.lower() or "convention" in entry.lower() for entry in substrings):
            substrings.append("new zealand")

        return substrings

    return pd.NA

def remove_special_terms(substring):
    special_terms = ["plaza", "street", "st", "road", "rd", "avenue", "close", "motorway", "highway",
                     "po", "PO", "new zealand|po", "office", "drive", "cor.", "ave.", "ave", "level", "lvl",
                     "box", "lv", "village", "vellage", "building", "bldg", "city", "albany"]

    pattern = r'\b(?:' + '|'.join(re.escape(term) for term in special_terms) + r')\b'

    substring = re.sub(pattern, '', substring, flags=re.IGNORECASE).strip()

    if "rubber" in substring.lower():
        substring = substring.replace("rubber", "").strip()

    if "tire" in substring.lower():
        substring = substring.replace("tire", "").strip()

    return substring

if __name__ == "__main__":
    json_file_path2 = "jobslabeled.json"
    df = pd.json_normalize(json.load(open(json_file_path2, 'r'))['profile']['professional_experiences'])

    # Focus on the 'location' column
    df['location'] = df['location'].str.lower()
    new_columns = df['location'].apply(split_and_create_columns).apply(pd.Series)

    # Label the new columns with subscripts and append numbers
    new_columns.columns = [f'subscript_{i+1}' for i in range(new_columns.shape[1])]

    # Concatenate the new columns to the original DataFrame
    df = pd.concat([df, new_columns], axis=1)

# Save df to a CSV file
df.to_csv('output_data.csv', index=False)

Job labeling completed. Labeled data saved to /content/jobslabeled.json
